In [1]:
# ignore SQlite warnings rel to decimal numbers in chinook db
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Import dependancies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, sessionmaker
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [3]:
#Create an engine for the chinook.sqlite database
engine = create_engine("sqlite:///Resources/10-Advanced-Data-Storage-and-Retrieval_3_Activities_11-Stu_Chinook_Resources_chinook.sqlite")
connection = engine.connect()

In [4]:
# Reflect database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect = True)

In [5]:
#Print the classes mapped out to the base
Base.classes.keys()

['albums',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoice_items',
 'invoices',
 'tracks',
 'media_types',
 'playlists']

In [6]:
# Save a reference to the invoices table as 'invoices'
Invoices = Base.classes.invoices
Items = Base.classes.invoice_items

In [7]:
#Create a database session object
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
x = pd.read_sql_query("SELECT * FROM invoices;", connection)
x

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06 00:00:00,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09 00:00:00,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14 00:00:00,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


In [9]:
#Design a query that lists all of the billing countries found in the invoices table.
list_country = pd.read_sql_query("SELECT DISTINCT BillingCountry FROM invoices", connection)
list_country

,BillingCountry
0,Germany
1,Norway
2,Belgium
3,Canada
4,USA
5,France
6,Ireland
7,United Kingdom
8,Australia
9,Chile


In [27]:
#Design a query that lists the invoices totals for each billing country and sort the output in descending order.

# for country in list_country:
#     if country == list_country,         


billing_total = pd.read_sql_query("SELECT BillingCountry, sum(Total) AS Total FROM invoices GROUP BY BillingCountry ORDER BY Total DESC;", connection)
# billing_total.sort_values(by = 'sum(Total)', ascending = False)

In [28]:
billing_total

,BillingCountry,Total
0,USA,523.06
1,Canada,303.96
2,France,195.10
3,Brazil,190.10
4,Germany,156.48
5,United Kingdom,112.86
6,Czech Republic,90.24
7,Portugal,77.24
8,India,75.26
9,Chile,46.62


In [11]:
#Design a query that lists all of the Billing Postal Codes for the USA.
postal_codes_usa = pd.read_sql_query("SELECT DISTINCT BillingPostalCode FROM invoices WHERE BillingCountry = 'USA'", connection)
postal_codes_usa

,BillingPostalCode
0,2113
1,94043-1351
2,98052-8300
3,95014
4,89503
5,53703
6,85719
7,76110
8,84102
9,32801


In [12]:
#"SELECT employees.emp_no, employees.emp_title_id, salaries.salary, titles.title FROM employees LEFT JOIN salaries ON employees.emp_no = salaries.emp_no LEFT JOIN titles on employees.emp_title_id = titles.title_id;

In [13]:
#Calculate the Item Totals sum(UnitPrice * Quantity) for the USA.
df2 =  pd.read_sql_query("SELECT \
invoice_items.InvoiceId, invoice_items.UnitPrice, invoice_items.Quantity, \
invoices.InvoiceId, invoices.BillingCountry \
FROM invoice_items \
LEFT JOIN invoices \
ON invoice_items.InvoiceId = invoices.InvoiceId \
WHERE invoices.BillingCountry = 'USA'", connection)
df2

,InvoiceId,UnitPrice,Quantity,InvoiceId,BillingCountry
0,5,0.99,1,5,USA
1,5,0.99,1,5,USA
2,5,0.99,1,5,USA
3,5,0.99,1,5,USA
4,5,0.99,1,5,USA
...,...,...,...,...,...
489,407,0.99,1,407,USA
490,408,0.99,1,408,USA
491,408,0.99,1,408,USA
492,408,0.99,1,408,USA


In [14]:
#Calculate the Item Totals sum(UnitPrice * Quantity) for the USA.
df3 =  pd.read_sql_query("SELECT sum(UnitPrice * Quantity) \
FROM invoice_items \
LEFT JOIN invoices \
ON invoice_items.InvoiceId = invoices.InvoiceId \
WHERE invoices.BillingCountry = 'USA'", connection)
df3

,sum(UnitPrice * Quantity)
0,523.06


In [15]:
#Calculate the invoice items totals sum(UnitPrice * Quantity) for each Billing Postal Code for the USA.

In [16]:
postal_code_billing =  pd.read_sql_query("SELECT invoice_items.InvoiceId, invoices.BillingPostalCode, sum(UnitPrice * Quantity) \
FROM invoice_items \
LEFT JOIN invoices \
ON invoice_items.InvoiceId = invoices.InvoiceId \
WHERE invoices.BillingCountry = 'USA' \
GROUP BY invoices.BillingPostalCode;", connection)
postal_code_billing

,InvoiceId,BillingPostalCode,sum(UnitPrice * Quantity)
0,112,10012-2612,37.62
1,5,2113,37.62
2,91,32801,39.62
3,17,53703,42.62
4,92,60611,43.62
5,70,76110,47.62
6,71,84102,43.62
7,39,85719,37.62
8,16,89503,37.62
9,113,94040-111,39.62
